- DB별 라이브러리가 필요하다. 즉 다운받아서 설치를 해야 한다. (파이썬과 DBMS을 연동하려면)
- 다른 DB라이브러리들도 사용법들은 거의 동일하다.

- 서버컴퓨터와 클라이언트 컴퓨터분리되어 네트워크로 연결되어있을때는  // cx-oracle
https://www.oracle.com/kr/database/technologies/instant-client/winx64-64-downloads.html
- 오라클 클라이언트 설치 사이트에서 클라이언트를 다운로드를 받아야한다.

- Mac에서 Oracle Instant Client - Homebrew
  - https://blog.naver.com/hanccii/222052865386
  
- window cx_Oracle 모듈 설치 방법
https://pypi.org/ 접속해서 cx_Oracle 검색
아나콘다 프롬프트 에 pip install cx-Oracle 입력 

or

https://anaconda.org/anaconda/repo 에서 검색 
conda install cx_oracle 입력 


# cx_Oracle 모듈
- 파이썬에서 오라클 DBMS와 연동는 다양한 함수를 제공하는 모듈
- Python DB API 2.0 표준을 따른다. 

## 참고
- [cx_Oracle 모듈 홈페이지](https://oracle.github.io/python-cx_Oracle/)
- [Python DB API](https://docs.python.org/ko/3/library/sqlite3.html)

## cx_Oracle 모듈 설치
- [설치 튜토리얼](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)
- 패키지 설치
```
conda install cx_oracle
pip install cx_Oracle --upgrade
```

## 코딩 절차
1. cx_Oracle 모듈 import

2. cx_Oracle모듈의 connect() 함수를 이용해 디비"연결" // (연결 : db와 연결하는 역할 /접속 scott 에 접속 하듯이)

3. Connection 객체의 cursor() 메소드를 이용해 Cursor객체 조회.
    - Cursor객체: sql문을 전송하고 select결과 조회 기능을 제공하는 객체

4. Cursor 의 execute() 메소드를 이용해 SQL문 전송

5. SELECT 의 경우 결과 조회.
    - 조회결과 한행씩 반환받아 사용
    - 조회결과 한번에 반환받아 사용

6. INSERT/DELETE/UPDATE의 경우  SQL 문 실행 후 Connection의 commit() 메소드를 이용해 Transaction 처리

7. Cursor, Connection 연결 닫기

### DB 연결
- connect() 함수를 이용하며 연결 후 Connection 객체를 받는다.
- 연결시 필요한 값
    - **host**        (ip 주소// 서버의 ip주소// 어느 컴퓨터 ///예를 들면 )어느나라에 갈지)
    - **port번호**    (어느 서버프로그램에 연결할지0~65535//어느 DB서버 //예를 들면)어떤항구에들어갈지)
    - **SID (DB이름)**   ( 오라클에서는 sid  다른 곳에선 db이름)(어느 DB에 접근할지)
    - **user name**()  
    - **password**

### Connection 주요 메소드
- commit(): 커밋
- rollback(): 롤백
- close(): 연결 닫기

In [2]:
# connect() 함수를 이용해서 연결. 연결(접속) 성공하면 connection객체를 반환
# connection객체: 연결정보(username,password,url*url은 host,port,sid를 묶어서)를 가지고 있는 객체 // 
#연결은 connect() 함수가 하는것 (connection 이라는 객체를 반환하고 그안의 메소드를 사용 ex) close()등)
import cx_Oracle
username = 'c##scott_join'
password = 'tiger'
url = "localhost:1521/XE"  #host:port/sid
conn = cx_Oracle.connect(username, password, url)  # 연결  /connect 함수를 이용해서 // conn(connection 객체이름) 객체를 받았다.
print(type(conn))
conn.close()  # 연결 해제 // 객체.메소드 

<class 'cx_Oracle.Connection'>


In [3]:
# username/pwd@url  연결방법 2
conn2 = cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")  #연결
print(type(conn2))
conn2.close()

<class 'cx_Oracle.Connection'>


In [3]:
import cx_Oracle

#### DSN (Data Source Name) 을 이용한 연결
- DSN: DBMS를 연결하기 위한 정보를 모아 놓은 것. (서로 다른 DBMS의 연결정보를 통일하기 위해서 방식)
- makedsn(host, port번호, SID) 함수 이용해 dsn을 생성한다.

In [4]:
username='c##scott_join'
password ='tiger'
host='localhost'
port=1521
sid='XE'

dsn=cx_Oracle.makedsn(host,port,sid)
conn=cx_Oracle.connect(username,password,dsn)


In [5]:
conn.close()  ## 연습한것

In [6]:
username='c##scott_join'
password ='tiger'
host='localhost'
port=1521
sid='XE'

#url = f'{username}/{password}@{host}:{port}/{sid}' 

dsn = cx_Oracle.makedsn(host,port,sid)  #makedsn 함수를 이용해서 url을 만듦// 묶음을 만들었다. 
print(dsn)
conn3 =cx_Oracle.connect(username,password,dsn) # data sause name
print(type(conn3))
conn3.close

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))
<class 'cx_Oracle.Connection'>


<function Connection.close>

In [ ]:
#with block  을 빠져나오면 자동으로 close()처리.  : 복습할때 with 구문 다시 보기 
with cx_Oracle.connect(username,password,dsn) as conn4:
    # DB관련 작업    conn4.xxx()
    pass



## Cursor 객체  
- SQL문을 전송하고 select결과를 조회하는 메소드들을 제공한다.
- Connection객체의 cursor() 메소드로 받아온다.
- cursor 현재위치// DB 에서의 data 의 현재 위치 

### Cursor의 주요 메소드
#### SQL 실행 메소드
- execute(sql) : 하나의 sql 문 실행. 
- executemany(sql): insert, update, delete 배치 처리  ** 동일한 쿼리를 값만 바꾸면서 여러개를 한번에 처리할때 쓴다.(ex: insert를 여러게 할때)
    - insert, update, delete는 다 처리후 conn.commit()으로 커밋 처리해야한다.
    

#### select 결과 조회 메소드
- execute()로 실행한 SQL이 select인 경우 다음 메소드로 조회한다.
- fetchall()  : 조회된 모든 행을 한번에 가져올 때 사용한다. 결과를 tuple들을 묶은 리스트로 반환
- fetchone()  : 호출시 마다 한행씩 반환한다. PK로 조회한 경우 많이 사용한다.
- fetchmany(n): n행만큼 조회한다. n기본값-100. 특정개수반큼 반복문을 이용해 가져올때 사용.

- 값을 갖고오는것을 fetch 라고 한다.

In [18]:
import cx_Oracle
from  pprint import pprint  #자료구조를 보기좋게 출력해주는 print함수.

In [28]:
#sql문 : 문자열로 작성. 대소문자 구분 안한다.//(sql의 keyword들은 보통 대문자로 작성, 컬럼명/테이블명은 소문자 *가독성 위해서)
#     :"" ';' 은 뒤에 붙이지 않는다. ""

select_sql = "SELECT emp_id, emp_name, job_id FROM emp"  # 디벨로퍼에서 작성해서 결과 확인후 하는게 편함 
insert_sql = "INSERT INTO emp VALUES (1500, 'g홍길동' , 'FI_ACCOUNT', 101, to_date('2021/02/15','yyyy/mm/dd'), 30000, 0.2, 100)"  #sql 문과 똑같이 작성한다고 생각하면됨
# 날짜 문자열이기 때문에 는 to_date로 해야한다.


conn = None  # connection을 대입할 변수 
cusor= None # cusor 객체를 대힙할 변수 

#insert 후에 select문을 조회 할것이다.
# 연결 -> cursor 객체 생성-> cursor실행(execute) -> 조회결과 -> conn.commit -> 연결닫기 cursor.close()/con.close()

try:
    #1 연결
    conn= cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")
    print('연결 완료')
    #2. cusor  객체 생성
    cursor = conn.cursor()
    #3.sql 실행
    cursor.execute(insert_sql)
    cursor.execute(select_sql)
    #4. select 조회 결과
    result = cursor.fetchall() #조회결과 한번에 반환. 한행 = tuple , 여러행 =  여러행의tuple 을 list로 묶어서 반환 
    print(type(result),type(result[0]))
    
    print(len(result))   #115 였는데 116으로 늘어남
    pprint(result)  # pprint 썻을때
    #5. insert/update/delete 를 실행한경우 commit 처리 
    conn.commit() # 커밋을 안하면 데이터에는 추가가 안됨// 그냥 조회는 가능하지만 
except Exception as e:
    print(e)
    
finally:
    #6. 연결 닫기 . - cursor/connect  다 끊기 
    if cursor !=None:
        cursor.close()
    if conn != None:  # None이 아니면 연결이 되서 conn에 connect 객체가 있을것이니까 ==None 이면 
        conn.close()
    

연결 완료
<class 'list'> <class 'tuple'>
116
[(100, 'Steven', None),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (116, 'Shelli', 'PU_CLERK'),
 (117, 'Sigal', 'PU_CLERK'),
 (118, 'Guy', 'PU_CLERK'),
 (119, 'Karen', 'PU_CLERK'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (125, 'Julia', 'ST_CLERK'),
 (126, 'Irene', 'ST_CLERK'),
 (127, 'James', 'ST_CLERK'),
 (128, 'Steven', 'ST_CLERK'),
 (129, 'Laura', 'ST_CLERK'),
 (130, 'Mozhe', None),
 (131, 'James', None),
 (132, 'TJ', 'ST_CLERK'),
 (133, 'Jason', 'ST_CLERK')

In [31]:
pprint(result) #하면 조회했던 값을 계속 사용

[(100, 'Steven', None),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (116, 'Shelli', 'PU_CLERK'),
 (117, 'Sigal', 'PU_CLERK'),
 (118, 'Guy', 'PU_CLERK'),
 (119, 'Karen', 'PU_CLERK'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (125, 'Julia', 'ST_CLERK'),
 (126, 'Irene', 'ST_CLERK'),
 (127, 'James', 'ST_CLERK'),
 (128, 'Steven', 'ST_CLERK'),
 (129, 'Laura', 'ST_CLERK'),
 (130, 'Mozhe', None),
 (131, 'James', None),
 (132, 'TJ', 'ST_CLERK'),
 (133, 'Jason', 'ST_CLERK'),
 (134, 'Michael', 'ST_CLERK'),
 (135, '

## placeholder(자리표시자)를 이용해 sql 실행
- SQL에 값이 들어갈 자리에 값을 대신할 문자 **`:순번` 또는 `:이름`** 를 넣고 SQL 실행시 값을 전달
> placehold 는 DBMS마다 다르다. 
- execute(SQL, placeholder에 들어갈 값)
    - 딕셔너리: placeholder의 이름-값 형식
    - 리스트나 튜플: placeholder 순서

In [32]:
import cx_Oracle
from pprint import pprint

In [40]:
#select_sql = "SELECT * FROM emp WHERE salary BETWEEN 10000 AND 20000"
select_sql = "SELECT * FROM emp WHERE salary BETWEEN :1 AND :2"  # 순번
#insert_sql = "INSERT INTO emp VALUES(:1,:2,:3,:4,:5)"
insert_sql = "INSERT INTO emp VALUES(:id, :name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)"
## dictionary 형태로 

conn = None
cursor = None

try:
    conn = cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")
    cursor= conn.cursor()
    # sql 문 실행
    
# varchar/char/nvarchar/nchar   - 문자열
# number (정수/ 실수) - int/ double
# date : datetime 객체   -- 아직 안배움  #datetime.datetime(2005, 9, 21, 0, 0)
# null : None 

    param_dict = {
        'id' : 2000,
        'name' :'이순신',
        'job_id':'FI_ACCOUNT',
        'mgr_id':101,
        'hire_date':'20210215',
        'salary': 1999,  # 실수일때는 정수 넣어도 무관하다.
        'comm_pct':None,
        'dept_id': 110
    }
    
    #insert 에 들어갈 값을 줌
    cursor.execute(insert_sql, param_dict)
    
    
    
    cursor.execute(select_sql, [15000,20000])
    result1= cursor.fetchall()
    pprint(result1)
    
    cursor.execute(select_sql, [2000,3000])
    result2= cursor.fetchall()
    print("15000~20000 사이 salary 조회")
    pprint(result2)
    
    
    conn.commit()
except Exception as e:
    print(e)
    
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

[(101,
  'Neena',
  'AD_VP',
  100,
  datetime.datetime(2005, 9, 21, 0, 0),
  17000.0,
  None,
  90),
 (102,
  'Lex',
  'AD_VP',
  100,
  datetime.datetime(2001, 1, 13, 0, 0),
  17000.0,
  None,
  90),
 (104,
  'Bruce',
  'IT_PROG',
  103,
  datetime.datetime(2007, 5, 21, 0, 0),
  18000.0,
  None,
  60),
 (109,
  'Daniel',
  'FI_ACCOUNT',
  108,
  datetime.datetime(2002, 8, 16, 0, 0),
  18000.0,
  None,
  100),
 (110,
  'John',
  'FI_ACCOUNT',
  108,
  datetime.datetime(2005, 9, 28, 0, 0),
  16400.0,
  None,
  100),
 (111,
  'Ismael',
  'FI_ACCOUNT',
  108,
  datetime.datetime(2005, 9, 30, 0, 0),
  15400.0,
  None,
  100),
 (112,
  'Jose Manuel',
  'FI_ACCOUNT',
  108,
  datetime.datetime(2006, 3, 7, 0, 0),
  15600.0,
  None,
  100)]
15000~20000 사이 salary 조회
[(116,
  'Shelli',
  'PU_CLERK',
  114,
  datetime.datetime(2005, 12, 24, 0, 0),
  2900.0,
  None,
  30),
 (117,
  'Sigal',
  'PU_CLERK',
  114,
  datetime.datetime(2005, 7, 24, 0, 0),
  2800.0,
  None,
  30),
 (118,
  'Guy',
  'PU

## SELECT  결과 조회

### fetchone()
- 조회결과 한행씩 반환한다.
- PK로 조회시 사용한다.   ex) emp_id 가 100 인 직원 찾을때
- 반환 type: tuple


### fetchall()
- 조회한 전체 결과셋을 반환한다.
- 한 row를 Tuple로 묶고 그 Tuple들을 List로 묶어서 반환한다.  
- fetchone pk 일때 쓰지만 all 써도 된다 하지만 결과에서 인덱스로 필요값을 갖고 올때 불편하다. r[0][1]   이런식으로 
-

### fetchmany(n)
- n 행씩 조회할 때 사용
- n 기본값 100        
- 원하는 행수 만큼 본다. page 개념으로 생각 

In [19]:
# emp_id(PK)로 직원 정보 조회
def select_emp_by_id(emp_id):
    sql = "SELECT * FROM emp WHERE emp_id =:1"    ## 플레이스 홀더라서 리스트로 넣어야함 하나라도 
    #conn, cursor = None,None 
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:  # 연결
        with conn.cursor() as cursor:      # 커서 객체 
            # sql실행
            cursor.execute(sql, [emp_id])
            result =  cursor.fetchone()      #1행만 조회, 조회결과가 없으면 None을 반환
            return result
        

In [20]:
r = select_emp_by_id(100)
if r != None:
    print(r, "id",r[0],'이름:',r[1])
    print("id",r[0],'이름:',r[1])  #튜플로 반환 되었으니까 
else:
    print("조회결과 없음")

(100, 'Steven', None, 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90) id 100 이름: Steven
id 100 이름: Steven


In [76]:
r1 = select_emp_by_id(100000)

#if r1 != None:
if r1:            #넌이면 false 니까
    print(r1)
else:
    print("조회결과 없음",r1)


조회결과 없음 None


In [63]:
sql = "SELECT * FROM emp order by emp_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)  # 이구문을 반복 하면 10개씩 계속 조회한다.
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)  # 첫번째꺼 10개 다음번 부터 10개씩 계속 조회한다.
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)
        
        result = cursor.fetchmany(20)
        print(len(result))
        print(result)  # 반복문으로 돌리는 것도 가능!

10
[(100, 'Steven', None, 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90), (101, 'Neena', 'AD_VP', 100, datetime.datetime(2005, 9, 21, 0, 0), 17000.0, None, 90), (102, 'Lex', 'AD_VP', 100, datetime.datetime(2001, 1, 13, 0, 0), 17000.0, None, 90), (103, 'Alexander', 'IT_PROG', 102, datetime.datetime(2006, 1, 3, 0, 0), 27000.0, None, 60), (104, 'Bruce', 'IT_PROG', 103, datetime.datetime(2007, 5, 21, 0, 0), 18000.0, None, 60), (105, 'David', 'IT_PROG', 103, datetime.datetime(2005, 6, 25, 0, 0), 14400.0, None, 60), (106, 'Valli', 'IT_PROG', 103, datetime.datetime(2006, 2, 5, 0, 0), 14400.0, None, 60), (107, 'Diana', 'IT_PROG', 103, datetime.datetime(2007, 2, 7, 0, 0), 12600.0, None, 60), (108, 'Nancy', 'FI_MGR', 101, datetime.datetime(2002, 8, 17, 0, 0), 24016.0, None, 100), (109, 'Daniel', 'FI_ACCOUNT', 108, datetime.datetime(2002, 8, 16, 0, 0), 18000.0, None, 100)]
10
[(110, 'John', 'FI_ACCOUNT', 108, datetime.datetime(2005, 9, 28, 0, 0), 16400.0, None, 100), (111, 'Ismael',

In [7]:
from pprint import pprint

### cursor를 for in 문을 이용해 select결과 조회
- select후 cursor는 Iterable 한 객체.
- cursor는 반복시마다 조회한 한행을 반환.

In [69]:
sql = "SELECT * FROM emp WHERE comm_pct IS NOT NULL ORDER BY emp_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)  
        #한행씩 조회해서 처리하는 경우   / cursor 가 iterable type 이다.
        for idx,row in enumerate(cursor, start=1): # 한번 반복할때마다 cursor.fetchone()을 실행
            print(f'{idx}. {row}')
            
            
#             r = cursor.fetchone()
#             while r != None:
#                 print(r)
#                 print('-------------')
#                 r= cursor.fetchone()

1. (100, 'Steven', None, 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90)
2. (145, 'John', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 14000.0, 0.4, 80)
3. (146, 'Karen', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 13500.0, 0.3, 80)
4. (147, 'Alberto', 'SA_MAN', 100, datetime.datetime(2005, 3, 10, 0, 0), 12000.0, 0.3, 80)
5. (148, 'Gerald', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 11000.0, 0.3, 80)
6. (149, 'Eleni', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 10500.0, 0.2, 80)
7. (150, 'Peter', 'SA_REP', 145, datetime.datetime(2007, 10, 15, 0, 0), 10000.0, 0.3, 80)
8. (151, 'David', 'SA_REP', 145, datetime.datetime(2005, 3, 24, 0, 0), 9500.0, 0.25, 80)
9. (152, 'Peter', 'SA_REP', 145, datetime.datetime(2005, 8, 20, 0, 0), 9000.0, 0.25, 80)
10. (153, 'Christopher', 'SA_REP', 145, datetime.datetime(2006, 3, 30, 0, 0), 8000.0, 0.2, 80)
11. (154, 'Nanette', 'SA_REP', 145, datetime.datetime(2006, 12, 9, 0, 0), 7500.0, 0.2, 80)
12. (155, '

In [8]:
sql = "SELECT job_id, sum(salary), round(avg(salary),2) FROM emp GROUP BY job_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)

[('FI_ACCOUNT', 153198, 13927.09),
 ('AD_VP', 34000, 17000),
 ('PU_CLERK', 10800, 2700),
 ('SH_CLERK', 55900, 3288.24),
 ('HR_REP', 6500, 6500),
 ('PU_MAN', 20100, 10050),
 ('AC_MGR', 12008, 12008),
 ('ST_CLERK', 47900, 2817.65),
 ('AD_ASST', 5500, 5500),
 (None, 48200, 6025),
 ('IT_PROG', 98320, 9832),
 ('SA_MAN', 61000, 12200),
 ('AC_ACCOUNT', 8300, 8300),
 ('FI_MGR', 24016, 24016),
 ('ST_MAN', 36400, 7280),
 ('MK_MAN', 13000, 13000),
 ('SA_REP', 250500, 8350),
 ('MK_REP', 6000, 6000),
 ('PR_REP', 10000, 10000)]


In [11]:
sql = 'SELECT e.emp_name, d.dept_name FROM emp e left join dept d ON e.dept_id = d.dept_id'
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)

[('Michael', 'Marketing'),
 ('Pat', 'Marketing'),
 ('홍길동', 'Marketing'),
 ('홍동', 'Marketing'),
 ('Den', 'Purchasing'),
 ('Alexander', 'Purchasing'),
 ('Shelli', 'Purchasing'),
 ('Sigal', 'Purchasing'),
 ('Guy', 'Purchasing'),
 ('Karen', 'Purchasing'),
 ('Susan', 'Human Resources'),
 ('Matthew', 'Shipping'),
 ('Adam', 'Shipping'),
 ('Payam', 'Shipping'),
 ('Shanta', 'Shipping'),
 ('Kevin', 'Shipping'),
 ('Julia', 'Shipping'),
 ('Irene', 'Shipping'),
 ('James', 'Shipping'),
 ('Steven', 'Shipping'),
 ('Laura', 'Shipping'),
 ('Mozhe', 'Shipping'),
 ('James', 'Shipping'),
 ('TJ', 'Shipping'),
 ('Jason', 'Shipping'),
 ('Michael', 'Shipping'),
 ('Ki', 'Shipping'),
 ('Hazel', 'Shipping'),
 ('Renske', 'Shipping'),
 ('Stephen', 'Shipping'),
 ('John', 'Shipping'),
 ('Joshua', 'Shipping'),
 ('Trenna', 'Shipping'),
 ('Curtis', 'Shipping'),
 ('Randall', 'Shipping'),
 ('Peter', 'Shipping'),
 ('Winston', 'Shipping'),
 ('Jean', 'Shipping'),
 ('Martha', 'Shipping'),
 ('Julia', 'Shipping'),
 ('Anthony', 

In [ ]:
#프롬프트에 netstat  을 치면 현재 사용한느 포트가 다조회가능
# netstat -an|findstr 1521 을 검색하면  1521이 사용중인인지 조회
# C:\app\Playdata\product\18.0.0\dbhomeXE\network\admin 에서 litener.ora 에서 보면 포트번호 조회가능 

## INSERT / UPDATE/ DELETE
- SQL 실행 후 Connection객체의 commit()을 호출 해야 최종적으로 적용된다.
- commit()
    - 최종 적용
- rollback()
    - 변경전 상태로 되돌리기

### 사용할 테이블
```sql
create table test(
    id varchar2(10),
    name varchar2(100),
    age number(3));
```

In [78]:
create_sql = "create table test(id varchar2(10),name varchar2(100),age number(3))"
insert_sql = "INSERT INTO test VAlUES (:1,:2,:3)"

with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        try:
            cursor.execute(create_sql)  #테이블 생성은 계속 되면 에러 나는게 당연하고 / insert는 된다 
        except Exception as e:
            print(e)
            #insert
        cursor.execute(insert_sql, [1,'이름1',20])
        cursor.execute(insert_sql, [1,'이름2',30])
        cursor.execute(insert_sql, [1,'이름3',10])
        conn.commit()

In [101]:
delete_sql = "DELETE FROM test WHERE name =:1"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(delete_sql, ['이름1'])
        conn.commit()


UPDATE : 테이블의 컬럼의 값을 수정
UPDATE 테이블명
SET    변경할 컬럼 = 변경할 값  [, 변경할 컬럼 = 변경할 값]
[WHERE 제약조건]

 - UPDATE: 변경할 테이블 지정
 - SET: 변경할 컬럼과 값을 지정
 - WHERE: 변경할 행을 선택. 

In [9]:
update_sql="UPDATE test SET name =:1"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(update_sql,['새이름'])
        conn.commit()


In [10]:
select_sql='select*from test order by id'
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(select_sql)
        pprint(cursor.fetchall())


[('1', '새이름', 20),
 ('1', '새이름', 30),
 ('1', '새이름', 10),
 ('a-1', '새이름', 30),
 ('a-2', '새이름', 20),
 ('a-2', '새이름', 20),
 ('a-3', '새이름', 33),
 ('a-3', '새이름', 33),
 ('a-4', '새이름', 40),
 ('a-4', '새이름', 40)]


## batch(일괄작업) 처리
- 한번에 메소드 호출로 다수 행을 처리한다.
- executemany(sql, placeholder에 전달할값)
    - placeholder에 전달할 값을 list로 묶어서 전달하면 sql문이 그 개수만큼 한번에 처리된다.

In [ ]:
insert_sql = 'INSERT INTO test VALUES(:1, :2, :3)'
param = [
    ['a-1','이름1',30],
    ['a-2','이름2',20],
    ['a-3','이름3',33],
    ['a-4','이름4',40]
    
]
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.executemany(insert_sql,param)
        conn.commit()

In [ ]:
# 전부 함수로 만들면됨

## TODO : 매개변수로 dept_id를 받아서 EMP 테이블에서 그 부서의 직원들의 salary를 두배 update하는 함수.

In [ ]:
UPDATE emp
SET salary= salary*2 
WHERE dept_id = 100

In [ ]:

def update_salary_by_dept_id(dept_id):
    update_sql = "UPDATE emp SET salary= salary*2 WHERE dept_id =:1"
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(update_sql,[dept_id])
            result = cursor.fetchall()
            return result

In [ ]:
r2 = update_salary_by_dept_id(80)
if r2 != None:
    print(r2)  
else:
    print('없음')

------------------------------위에꺼 틀린거 -------------------------------
 - 조회 하는거는 무조건 select 로 해서 봐야함.. 위에서는 조회하는 거까지 해서 틀렸음 

In [5]:
import cx_Oracle
def update_salary_by_dept_id(dept_id):
    sql = "UPDATE emp SET salary= salary*2 WHERE dept_id =:1"  #여기까지 맞음
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql,[dept_id])
            conn.commit()  # 조회 하는거는 무조건 select문으로 해서 봐야함 파이썬에서 볼때

In [6]:
update_salary_by_dept_id(80)

## TODO : 매개변수로 job_id를 받아서 그 업무를 하는 직원들을 EMP 테이블에서 삭제하는 함수.

In [ ]:
딜리트

In [8]:
def delete_job_id(job_id):
    if job_id =! None:
        delete_sql = "DELETE FROM emp WHERE job_id=:1"
    else:
        delete_sql = "DELETE FROM emp WHERE job_id is null"  # null 인 데이터 삭제 
        
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            if jon_id != None:
                cursor.execute(delete_sql,[job_id])
                conn.commit()
            else:
                cursor.execute(sql)  # 잡 아이디가 없으니까
           

In [ ]:
delete_job_id('AD_VP')   # 복습할때 해보기 `

In [ ]:
delete_job_id(None) # job_id가 null 인애들 지우라는 함수이다. 실행했는데 삭제가 되지 않았다.

예. 두번째 sql을 보시면 placeholder가 없잖아요. 그래서 sql만 넣어서 실행한 거에요.



--------------------------------------밑에꺼 틀린거------------------------------
- ## 조회가 해서 값을 여기서 보는 함수가 아닌데  조회를 해서 안 됐던것임

In [ ]:
r2 = delete_job_id('PU_MAN')
if r2 != None:
    print(r2)  
else:
    print('없음')  ## 조회가 안되는 함수를 조회하려고 해서 안 됐던것임

In [ ]:
# 정지 눌러도 아스타리스크가 안없어진다. 

# TODO : 매개변수로 직원 이름을 입력받아 EMP 테이블에서 그  직원들의 전체 정보를 조회결과를 출력하는 함수.   ------조회하는것과 수정하는것을 구분해서 공부다시 하기


In [12]:
import cx_Oracle
def select_emp_by_name(emp_name):
    #sql= "SELECT * FROM emp WHERE emp_name=:1"
    sql = "SELECT * FROM emp WHERE emp_name LIKE '%'||:name||'%'"
   # sql = "SELECT * FROM emp WHERE emp_name LIKE '%'John'%'"  ==> 이상태로 입력 받게 된다 db 에서 
   # sql = "SELECT * FROM emp WHERE emp_name LIKE '%John%'"  ==> 이걸 원했는데 
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            #cursor.execute(sql,[emp_name])
            cursor.execute(sql,{"name":emp_name})  #---> :name 으로 했기 때문에 딕셔너리로 잆력
            result = cursor.fetchall()
            return result

In [14]:
r= select_emp_by_name('Jo')
print(r)

[(110, 'John', 'FI_ACCOUNT', 108, datetime.datetime(2005, 9, 28, 0, 0), 32800.0, None, 100), (112, 'Jose Manuel', 'FI_ACCOUNT', 108, datetime.datetime(2006, 3, 7, 0, 0), 31200.0, None, 100), (139, 'John', 'ST_CLERK', 123, datetime.datetime(2008, 2, 6, 0, 0), 2700.0, None, 50), (140, 'Joshua', None, 123, datetime.datetime(2008, 2, 6, 0, 0), 2500.0, None, 50), (145, 'John', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 28000.0, 0.4, 80), (176, 'Jonathon', 'SA_REP', 149, datetime.datetime(2006, 3, 24, 0, 0), 17200.0, 0.2, 80)]


In [ ]:
#       '%'||:name||'%'" =====>>> //'%'||'John'||'%'"   이렇게 되야 세개를 합해서 '%John%' 이렇게 바뀐다.

# TODO: 사용자로 부터 직원정보를 입력받아 EMP 테이블에 입력한다.
- emp_id, emp_name, job_id, mgr_id, hire_date, salary, comm_pct, dept_id 를 하나씩 입력받는다.
- job_id, mgr_id,  comm_pct, dept_id 을 입력하지 않은 경우 NULL 을 넣는다. (placehold 사용시 None을 대입)
- 입력된 값을 EMP 테이블에 insert 한다.
- 주의 job_id, mgr_id, dept_id는 FK 컬럼이므로 부모테이블의 PK값을 넣어야 한다.

In [ ]:
input("job_id")

In [19]:
def insert_emp():
    # 사용자로부터 insert 값 q받고 검증 작업
    
    emp_id = int(input("직원ID:"))
    emp_name = input("이름:")
    job_id = input('업무ID')
    if len(job_id.strip()) == 0:
        job_id = None
        
        
    try:
        mgr_id = int(input("상사ID"))  # 정수여야만 하니까 //정수가 아니면 예외 처리 
    except:
        mgr_id = None          # 널을 가질수 있으니까  잘못입력하면 null 이 되게 하려고 
        
    hire_date =  input("입사일(형식:yyyymmdd):")  # 입력하는 형식을 맞춰서 오류 피하기 위해
    salary =  int(input("급여:"))   #  notnull 이라서 잘못된 값을 넣었을때 null 을 넣을 수 없어서 잘못된값일때 에러를 내서 끝내려고한거다.
    
    
    try:
        comm_pct = float(input("커미션"))
        if comm_pct >= 1:
            raise Exception()
    except:
        comm_pct = None
    
    try: 
        dept_id = int(input("부서ID:"))
        
    except:
        dept_id = None
        
        
    param = {
        "emp_id":emp_id,
        "emp_name" : emp_name,
        "job_id": job_id,
        "mgr_id": mgr_id,
        "hire_date": hire_date,
        "salary":salary,
        "comm_pct": comm_pct,
        "dept_id": dept_id
    }
        
    sql = "INSERT INTO emp VALUES(:emp_id, :emp_name, :job_id, :mgr_id, to_date(:hire_date,'yyyymmdd'), :salary, :comm_pct, :dept_id)"
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql,param)
            conn.commit()
            print("insert완료")
            
        
        #입력해보고 insert 확인해보기// 핸드폰 사진 보기 
        

In [20]:
insert_emp()

직원ID:3220
이름:박명수
업무IDFI_ACCOUNT
상사ID
입사일(형식:yyyymmdd):20200815
급여:10000
커미션
부서ID:
insert완료


다른 연산 join 같은것도 연습해보기 